# Using AutoGen with motleycrew

Microsoft AutoGen is one of the most popular multi-agent frameworks. Motleycrew supports interaction with autogen in both directions: firstly, you can wrap an AutoGen chat as a motleycrew tool, which you can then give to any supported agent; secondly, you can give any motleycrew tool (which includes agents used as tools) to an AutoGen agent. 

The two options are described in detail below

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Integrating an AutoGen chat into motleycrew

If you want to use an AutoGen chat (say one from their many wonderful examples, or one you have already) as part of a motleycrew setup, you can wrap it as a tool, which you can then give to any other motleycrew-supported agent.

Let's create an AutoGen chat for automatic problem solving.
The code is taken from the example here: https://microsoft.github.io/autogen/docs/notebooks/agentchat_groupchat

In [1]:
!pip install pyautogen
import autogen
import os

llm_config = {
    "config_list": [{"model": "gpt-4-turbo", "api_key": os.environ["OPENAI_API_KEY"]}],
    "cache_seed": None,
}

In [3]:
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "examples/data/groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="TERMINATE",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

If we were using plain AutoGen, we'd start the chat with something like this:
```
user_proxy.initiate_chat(
    manager, message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."
)
```

You see, the chat accepts an input and returns an output, just like a tool.  
Because of that, for using the chat in motleycrew, we can utilize the built-in AutoGenChatTool. Its prompt can either be a template or a plain string. Here we are creating a tool that searches arXiv for recent papers.

In [4]:
from motleycrew.tools.autogen_chat_tool import AutoGenChatTool

knowledge_retrieval_tool = AutoGenChatTool(
    name="retrieve_knowledge_by_topic",
    description="Search arxiv for the latest paper on a given topic "
                "and find its potential applications in software.",  # will be used in the prompt of the future agent that will use the tool
    prompt="Find a latest paper about {topic} on arxiv "
            "and find its potential applications in software.",  # this is the initial prompt for the AutoGen chat itself
    initiator=user_proxy,
    recipient=manager,
)

We can now give the tool to any agent and solve tasks with it.

In [5]:
from motleycrew import MotleyCrew
from motleycrew.agents.langchain.tool_calling_react import ReActToolCallingAgent

crew = MotleyCrew()
writer = ReActToolCallingAgent(tools=[knowledge_retrieval_tool])

In [6]:
from motleycrew.tasks import SimpleTask

blog_post_task = SimpleTask(
        crew = crew,
        name="Produce blog post on the applications of latest advancements related to GPT-4",
        description="Using the insights provided by searching research papers, develop an engaging blog "
                    "post that highlights the most significant advancements on GPT-4 ant their applications.\n"
                    "Your post should be informative yet accessible, catering to a tech-savvy audience.\n"
                    "Make it sound cool, avoid complex words so it doesn't sound like AI. "
                    "Create a blog post of at least 4 paragraphs.",
        agent=writer,
    )

In [7]:
crew.run()

User_proxy (to chat_manager):

Find a latest paper about GPT-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
User_proxy (to chat_manager):



--------------

[Task(status=done)]

In [8]:
from IPython.display import display, Markdown
display(Markdown(blog_post_task.output))

Final Answer: Welcome to the Future: Exploring the Frontiers of GPT-4

In the rapidly evolving world of artificial intelligence, GPT-4 stands out as a beacon of innovation and potential. Recent research, particularly a fascinating study titled "Observational Scaling Laws and the Predictability of Language Model Performance," sheds light on groundbreaking advancements that could redefine how we interact with technology.

GPT-4 isn't just about understanding or generating text; it's about predicting and optimizing performance in ways previously deemed impossible. The study introduces predictive benchmarking tools that could revolutionize the development of AI models. Imagine software that can anticipate the performance of language models like GPT-4 based on various inputs, streamlining the development process and enhancing efficiency without the need for extensive testing.

But the implications extend beyond mere prediction. The research discusses resource optimization algorithms that could significantly reduce the costs associated with AI training. These algorithms utilize a "capability space" to optimize the compute resources needed, ensuring that AI development is not only faster but also more economical.

Moreover, GPT-4 could lead to the creation of automated model scaling software. Such tools would adjust AI capabilities based on available resources, making cutting-edge technology accessible to a broader range of users and organizations, regardless of their fluctuating resource availability.

The integration of these advancements into AI development frameworks could also empower developers with better tools for debugging and performance analysis, enhancing the robustness and accuracy of AI systems. This is not just about making AI smarter; it's about making AI development smarter, more intuitive, and infinitely more creative.

As we stand on the brink of these exciting developments, GPT-4 is not just a tool but a transformational force in the tech landscape, promising to bring about a new era of innovation and efficiency. Stay tuned, because the future of AI is here, and it's more promising than ever!

The writer agent used the AutoGen chat tool to retrieve data for the post!



## Using any motleycrew-supported tools with Autogen agents

Now let's do it the other way around: give tools (or agents, for that matter) from motleycrew to AutoGen agents. The code is based on https://microsoft.github.io/autogen/docs/tutorial/tool-use

In [24]:
from motleycrew.tools import MotleyTool
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = MotleyTool.from_supported_tool(DuckDuckGoSearchRun())  # Any tools or even motleycrew's agents can be converted to MotleyTool like this!

# Let's first define the assistant agent that suggests tool calls.
assistant = autogen.ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can provide useful and up-to-date data using web search. "
    "Return 'TERMINATE' when the task is done.",
    llm_config=llm_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = autogen.ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Make an autogen-compatible tool from the MotleyTool.
autogen_tool = search_tool.to_autogen_tool()

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="search_tool", description="Web search tool")(autogen_tool)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="search_tool")(autogen_tool)

<function motleycrew.tools.tool.MotleyTool.to_autogen_tool.<locals>.autogen_tool_fn(input: str) -> str>

In [25]:
chat_result = user_proxy.initiate_chat(assistant, message="What was the first computer?")

User (to Assistant):

What was the first computer?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_Na0NhkbZVtS7cnme7XRqxqUS): search_tool *****
Arguments: 
{"input":"first computer"}
****************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_tool...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_Na0NhkbZVtS7cnme7XRqxqUS) *****
ENIAC, the first programmable general-purpose electronic digital computer, built during World War II by the United States and completed in 1946. The project was led by John Mauchly, J. Presper Eckert, Jr., and their colleagues. ENIAC was the most powerful calculating device built to that time. Learn about the origins and evolution of computers, from the first mechanical and elec

In [ ]:
final_result = chat_result.summary

The agent used the search tool!